In [1]:
import pandas as pd
import re
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split

In [58]:
def GenerateXY(csv):
  # prepare dataframe
  df = pd.read_csv(csv)
  df.dropna(subset=['type'], inplace = True)
  df.drop(df[df['type'] == 'unknown'].index, inplace=True)
  df.reset_index(drop=True, inplace=True)
  
  # target values
  y = pd.DataFrame({'result' : np.where(df['type'] == 'fake', 1, 0)}) #[1, 0] = [fake, not fake]
  
  words = Counter()
  words.update("num".split())
  words.update("date".split())
  words.update("url".split())
  words.subtract(words)
  
  frames = []
  for i in range(len(df)):
    wordcopy = words.copy()
    for word in df.loc[i, "content"].split():
      if word == "<num>":
        wordcopy.update("num".split())
      if word == "<url>":
        wordcopy.update("url".split())
      if word == "date":
        wordcopy.update("date".split())
    frames += [pd.DataFrame(wordcopy, index=[i])]
    
  X = pd.concat(frames)
  
  return X
  
X, y = GenerateXY("../cleaned_news_sample.csv")

X.to_csv('X.csv', index=False)
y.to_csv('y.csv', index=False)

,num,date,url
0,1,0,0
1,9,0,3
2,7,0,2
3,4,0,0
4,9,0,0
...,...,...,...
227,1,0,0
228,6,0,1
229,1,0,0
230,20,0,1


In [ ]:
def Split(X_path, y_path):
    # usecols = ['content', 'type']
    # df = pd.read_csv(csv_path, usecols=usecols)
    
    df_X = pd.read_csv(X_path)
    df_y = pd.read_csv(y_path)
    X_train, X_test, y_train, y_test = train_test_split(df_X,
                                                        df_y,
                                                        train_size = 0.8,
                                                        random_state = 0,
                                                        shuffle = False,
                                                        stratify = None)

    X_test, X_val, y_test, y_val = train_test_split(X_test,
                                                    y_test,
                                                    test_size= 0.5,
                                                    random_state = 0,
                                                    shuffle = False,
                                                    stratify = None)

    return X_train, X_test, X_val, y_train, y_test, y_val
X_train, X_test, X_val, y_train, y_test, y_val = Split('X.csv', 'y.csv')